## 示例：调用天气API

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

True

In [2]:
from langchain_openai import ChatOpenAI

model = 'gpt-3.5-turbo-0613'
llm = ChatOpenAI(model=model)

In [3]:
import requests
import json
import os
from langchain_core.tools import tool

# 这里需要替换为你自己的OpenWeatherMap API密钥
open_weather_key = os.getenv("OPEN_WEATHER_API_KEY")

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，
                注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return: OpenWeather API查询即时天气的结果，具体URL请求地址为: https://api.openweathermap.org/data/2.5/weather
             返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"
    # Step 2.设置查询参数
    params = {
        "q": loc,
        "appid": open_weather_key,  # 输入API key
        "units": "metric",  # 使用摄氏度而不是华氏度
        "lang": "zh_cn"  # 输出语言为简体中文
    }
    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

tools = [get_weather]
llm_with_tools = llm.bind_tools(tools)

In [5]:
query = "请问北京今天天气如何？"
llm_with_tools.invoke(query).tool_calls

[{'name': 'get_weather',
  'args': {'loc': 'Beijing'},
  'id': 'call_VPuTJdrss1AHpOCyhQfRlUsr',
  'type': 'tool_call'}]

In [6]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_weather": get_weather}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

[HumanMessage(content='请问北京今天天气如何？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qcmGczIBXx1I9UNBPLjbvomH', 'function': {'arguments': '{"loc":"Beijing"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 195, 'total_tokens': 210, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6330ae9-9562-444e-85cf-eb321b28612a-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_qcmGczIBXx1I9UNBPLjbvomH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 195, 'output_tokens': 15, 'total_tokens': 210, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 803, "main": "Clou

In [8]:
llm_with_tools.invoke(messages).content

'北京今天的天气情况如下：\n- 天气状况：多云\n- 温度：21.94°C\n- 最高温度：21.94°C\n- 最低温度：21.94°C\n- 湿度：41%\n- 风速：3.33 m/s\n\n请问还有什么可以帮助您的吗？'